In [1]:
import os
import numpy as np
import scipy as sp
from scipy import stats
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from gc import collect

In [2]:
def write_prediction_to_file(prediction):
    header = "id,nextvisit"
    with open("submission.csv", "w") as f:
        print(header, file=f)
        for idx, p in enumerate(prediction, 1):
            print(f"{idx}, {p}", file=f)

In [3]:
def visits2weekdays(visits):
    return np.apply_along_axis(lambda x: (x - 1) % 7 + 1, 0, visits)

def visits2history(visits):
    history = np.zeros(PERIOD_LENGTH, dtype=int)
    history[visits-1] = 1
    return history

In [4]:
PERIOD_LENGTH = 1098

with open('./train.csv', 'r') as f:
    data = pd.read_csv(f)
    data.visits = data.visits.apply(lambda x: np.fromstring(x, dtype=int, sep=" "))
    
train_data = np.array([v[:-1] for v in data.visits])
test_data = np.array([v[-1] for v in data.visits])
collect()

0

In [52]:
train_df = pd.DataFrame(columns=["visits", "weekdays", "history"])
train_df.visits = train_data
train_df.weekdays = train_df.visits.apply(lambda v: visits2weekdays(v))
train_df.history = train_df.visits.apply(lambda v: visits2history(v))
collect()

21

In [8]:
train_df.head()

,visits,weekdays,history
0,"[30, 84, 126, 135, 137, 179, 242, 342, 426, 45...","[2, 7, 7, 2, 4, 4, 4, 6, 6, 1, 5, 7, 7, 6, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[24, 53, 75, 134, 158, 192, 194, 211, 213, 238...","[3, 4, 5, 1, 4, 3, 5, 1, 3, 7, 6, 4, 5, 5, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[51, 143, 173, 257, 446, 491, 504, 510, 559, 6...","[2, 3, 5, 5, 5, 1, 7, 6, 6, 7, 5, 7, 6, 2, 7, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[1, 20, 22, 92, 124, 149, 211, 335, 387, 390, ...","[1, 6, 1, 1, 5, 2, 1, 6, 2, 5, 7, 5, 6, 5, 2, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[34, 51, 56, 106, 110, 121, 163, 233, 266, 275...","[6, 2, 7, 1, 5, 2, 2, 2, 7, 2, 2, 2, 2, 5, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [57]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, Embedding
from keras.utils import to_categorical

# one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

In [102]:
train_x = [row[-25:-1] for row in train_df.weekdays] 
train_y = [row[-1] for row in train_df.weekdays]
train_x = np.stack(train_x, axis=0)-1
train_y = np.stack(train_y, axis=0)-1

In [83]:
train_x[0], train_y[0]

(array([1, 3, 3, 3, 5, 5, 0, 4, 6, 6, 5, 4, 1, 3, 1, 1, 2, 5, 3, 6, 3, 0,
        1, 2]), 3)

In [103]:
#train_x = to_categorical(train_x, num_classes=7)
train_y = to_categorical(train_y, num_classes=7)

In [104]:
model = Sequential()
model.add(Embedding(7, 16, input_length=24))
model.add(LSTM(100))
model.add(Dense(7, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 24, 16)            112       
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               46800     
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 707       
Total params: 47,619
Trainable params: 47,619
Non-trainable params: 0
_________________________________________________________________
None


In [105]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [106]:
model.fit(train_x, train_y, batch_size=64, epochs=500, verbose=2)

Epoch 1/500
 - 45s - loss: 1.8444 - acc: 0.2570
Epoch 2/500
 - 46s - loss: 1.8306 - acc: 0.2642
Epoch 3/500
 - 46s - loss: 1.8279 - acc: 0.2657
Epoch 4/500
 - 46s - loss: 1.8266 - acc: 0.2657
Epoch 5/500
 - 46s - loss: 1.8254 - acc: 0.2667
Epoch 6/500
 - 46s - loss: 1.8243 - acc: 0.2672
Epoch 7/500
 - 46s - loss: 1.8232 - acc: 0.2680
Epoch 8/500
 - 47s - loss: 1.8218 - acc: 0.2683
Epoch 9/500
 - 47s - loss: 1.8203 - acc: 0.2689
Epoch 10/500
 - 47s - loss: 1.8187 - acc: 0.2703
Epoch 11/500
 - 47s - loss: 1.8161 - acc: 0.2717
Epoch 12/500
 - 46s - loss: 1.8133 - acc: 0.2739
Epoch 13/500
 - 46s - loss: 1.8096 - acc: 0.2763
Epoch 14/500
 - 46s - loss: 1.8056 - acc: 0.2778
Epoch 15/500
 - 47s - loss: 1.8011 - acc: 0.2814
Epoch 16/500
 - 47s - loss: 1.7963 - acc: 0.2840
Epoch 17/500
 - 47s - loss: 1.7913 - acc: 0.2866
Epoch 18/500
 - 47s - loss: 1.7864 - acc: 0.2903
Epoch 19/500
 - 47s - loss: 1.7818 - acc: 0.2938
Epoch 20/500
 - 47s - loss: 1.7771 - acc: 0.2964
Epoch 21/500
 - 47s - loss: 1

KeyboardInterrupt: 

In [107]:
test_x = [row[-24:] for row in train_df.weekdays]
test_x = np.stack(test_x, axis=0)-1
test_y = np.array([row[-1] for row in train_df.weekdays])-1

pred = model.predict_classes(test_x ,verbose=2)

In [108]:
pred += 1

In [109]:
write_prediction_to_file(pred)

In [97]:
from sklearn.metrics import accuracy_score

accuracy_score(test_y, pred)

0.2736133333333333

In [90]:
test_x = np.stack(test_x, axis=0)
test_x.shape

(300000, 24)

In [ ]:
stats.mode(train_df.weekdays[0])

In [ ]:
DAYS_IN_WEEK = 7
WEEK_IN_MONTH = 4
DAYS_IN_MONTH = DAYS_IN_WEEK * WEEK_IN_MOTH
MONTH_IN_YEAR = 12